<a href="https://colab.research.google.com/github/wpfkcm33/korean_hate_speech/blob/main/bert_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import HfApi, notebook_login

# Hugging Face 계정으로 로그인
notebook_login()

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd

model_name = 'Doowon96/roberta-base-finetuned-hate_speech-best'

num_labels = 3
# {HuggingFace Model Hub 사용자 아이디}/{push_to_hub_model_id에서 설정한 값}
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# GPU 사용 설정 (가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset
import pandas as pd

# val 데이터셋 로드
val_dataset = load_dataset('Doowon96/hate_speech_labeled', split='validation')
val_df = pd.DataFrame(val_dataset)

# 결측치 확인
print(val_df.isna().sum())

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

comments               0
contain_gender_bias    0
bias                   0
hate                   0
dtype: int64


In [5]:
import torch
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

In [8]:
from datasets import Dataset

# Pandas DataFrame을 Hugging Face의 Dataset으로 변환
dataset_val = Dataset.from_pandas(val_df)

# 카테고리를 정수로 매핑하는 딕셔너리
category_to_id = {'none': 0, 'offensive': 1, 'hate': 2}

def preprocess_function(examples):
    # 제목을 토크나이즈
    tokenized_titles = tokenizer(examples['comments'], truncation=True, padding=True)

    # 카테고리 레이블을 정수로 변환
    tokenized_titles['labels'] = [category_to_id[category] for category in examples['hate']]

    return tokenized_titles

# 전처리 함수 적용
encoded_dataset_val = dataset_val.map(preprocess_function, batched=True)

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

In [9]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch.nn.utils.rnn import pad_sequence
import torch

def collate_fn(batch):
    # input_ids와 attention_mask를 텐서로 변환
    input_ids = pad_sequence([torch.tensor(item['input_ids']) for item in batch], batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence([torch.tensor(item['attention_mask']) for item in batch], batch_first=True, padding_value=0)

    # labels를 텐서로 변환
    labels = torch.tensor([item['labels'] for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }


def predict(model, data_loader):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch in tqdm(data_loader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            predictions.extend(preds.cpu().numpy())
            actuals.extend(batch['labels'].cpu().numpy())

    return predictions, actuals

# DataLoader 설정

data_loader_val = DataLoader(encoded_dataset_val, batch_size=32, collate_fn=collate_fn, shuffle=False)

# 예측 수행
predictions, actuals = predict(model, data_loader_val)


  0%|          | 0/15 [00:00<?, ?it/s]

In [10]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


In [11]:
from evaluate import load

# 정확도 평가 메트릭 로드
accuracy_metric = load('accuracy')

# 모델의 정확도 계산
accuracy_score = accuracy_metric.compute(predictions=predictions, references=actuals)
print("Accuracy:", accuracy_score)


Accuracy: {'accuracy': 0.6369426751592356}
